In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import pandas as pd
from datasets import Dataset


In [2]:
# 定义模型名称
model_name = "/root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B" 

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'right'  # 确保 padding_side 为 'right'

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map={"": 0},  # 将模型加载到第一个 GPU
    trust_remote_code=True  # 确保加载自定义代码
)

# LoRA配置
lora_config = LoraConfig(
    task_type="CAUSAL_LM",  # 微调模型为自回归模型
    r=16,  # LoRA 低秩分解的秩
    lora_alpha=32,  # LoRA 缩放因子
    target_modules=["q_proj", "v_proj"],  # 根据 Qwen2 的架构指定目标模块
    lora_dropout=0.05,  # Dropout 概率
    bias="none",  # 不训练 bias
    init_lora_weights=True,  # 初始化 LoRA 层权重
    inference_mode=False  # 允许训练
)

# 将LoRA配置应用到模型
model = get_peft_model(model, lora_config)

# 定义训练参数
training_arguments = TrainingArguments(
    output_dir="./DeepSeek-R1-Distill-Qwen-7B_LoRA",
    eval_strategy="no",  # 禁用评估
    optim="paged_adamw_8bit",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    log_level="debug",
    save_strategy="epoch",
    logging_steps=80,
    learning_rate=1e-4,
    fp16=False,  # 根据硬件支持选择
    bf16=False,  # 根据硬件支持选择
    num_train_epochs=1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
)

# 数据预处理
def process_func(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"User: {example['instruction']} {example['input']}\n\n", add_special_tokens=False)
    response = tokenizer(f"Assistant: {example['output']}{tokenizer.eos_token}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:  # 截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# 加载数据
df = pd.read_json('./huanhuan.json')
ds = Dataset.from_pandas(df)
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)

# 创建训练器
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [3]:
ds[:10]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。',
  '嬛妹妹，我虽是一介御医，俸禄微薄，可是我保证会一生一世对你好，疼爱你，保护你，永远事事以你为重。本来没半月一次到府上去请脉，能够偶尔见一次妹妹的笑靥，已经心满意足了，可谁知——而且我也知道，妹妹心里是不愿意去殿选的。',
  '实初虽然唐突了妹妹，却是真心实意地希望妹妹不要去应选，这不仅仅是因为我心里一直把妹妹当成……其实更是因为甄伯父曾经救过家父的性命。',
  '可是我父亲当年被诬，起因也是因为后宫争斗，不能独善其身。一介御医尚且如此，何况妹妹如果被选中的话，会身在其中啊。',
  '在京里休息了这些日子，早已经调养过来了。',
  '是啊。可是我总还想着我们一起长大的情分呢。诶？妹妹今日打扮得好生素净，可是细看起来还是个美人坯子，怎么看都是好的。',
  '你是谁？',
  '大理寺少卿，也不是什么高官。'],
 'input': ['', '', '', '', '', '', '', '', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。',
  '你们俩话太多了，我该和温太医要一剂药，好好治治你们。',
  '出来走走，也是散心。',
  '实初哥哥这么说，就枉顾我们一直以来的兄妹情谊了，嬛儿没有哥哥，一直把你当作自己的亲哥哥一样看待，自然相信哥哥会待妹妹好的——自然了，以后有了嫂子，你也会对嫂子更好。',
  '我们两家是世交，昔年恩义不过是父亲随手之劳，不必挂怀。',
  '实初哥哥的话我都明白，只是我不去应选，迟早也是玉娆，家中无子，女儿还能不孝吗？',
  '如今你住在自己京城的宅子里，不比从前住在外祖家，一墙之隔，见面也方便。',
  '沈大美人差矣，姐姐出落得这么标致，皇上见过必定会念念不忘。',
  '家父是大理寺少卿甄远道。',
  '凡事不论官位高低，只论个理字。']}

In [4]:
# 使用训练前的模型回答问题，max_length可以调整这个模型的输出token的长度,up主建议差不多2048基本够了
def generate_response(model, tokenizer, prompt, max_length=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 回答模板
def answer_template(prompt):
    return f"User: {prompt}\n\nAssistant:"

# 示例问题
question = "大理寺少卿，也不是什么高官。"
print("Answer before training:")
print(generate_response(model, tokenizer, answer_template(question)))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer before training:
User: 大理寺少卿，也不是什么高官。

Assistant: 那么，大理寺少卿是一个不太重要的职位，属于比较低级的官员。在中国古代，官员的等级是根据职责和权力来划分的，大理寺少卿可能主要负责一些文书工作和地方管理。

大理寺在古代是负责宗教事务和司法的机构，少卿作为其中的官员，可能有一定的职责范围。不过，大理寺的官员 hierarchy 可能比现代更为复杂，涉及多个部门和层级。

总的来说，大理寺少卿不是一个非常显赫的职位，更多地


In [5]:
# 开始训练
trainer.train()
# # 保存模型
trainer.save_model("./DeepSeek-R1-Distill-Qwen-7B_LoRA")

Currently training with a batch size of: 2
***** Running training *****
  Num examples = 3,729
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 466
  Number of trainable parameters = 5,046,272


Step,Training Loss
80,4.513000
160,3.658000
240,3.659800
320,3.538500
400,3.468700


Saving model checkpoint to ./DeepSeek-R1-Distill-Qwen-7B_LoRA/checkpoint-466
loading configuration file /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.0",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 152064
}

tokenizer config file saved in ./DeepSeek-R1-Distill-Qwen-7B_LoRA/checkpoint-466/tokenizer_config.json
Specia

In [9]:
# 使用回答模板生成回答
question = "小姐，别的秀女都在求中选"
print("\nAnswer with template:")
print(generate_response(model, tokenizer, answer_template(question)))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Answer with template:
User: 小姐，别的秀女都在求中选

Assistant: 那好，我来选吧。
